### Problem 1 (50 points) 

Vapor-liquid equilibria data are correlated using two adjustable parameters $A_{12}$ and $A_{21}$ per binary
mixture. For low pressures, the equilibrium relation can be formulated as:

$$
\begin{aligned}
p = & x_1\exp\left(A_{12}\left(\frac{A_{21}x_2}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{water}^{sat}\\
& + x_2\exp\left(A_{21}\left(\frac{A_{12}x_1}{A_{12}x_1+A_{21}x_2}\right)^2\right)p_{1,4 dioxane}^{sat}.
\end{aligned}
$$

Here the saturation pressures are given by the Antoine equation

$$
\log_{10}(p^{sat}) = a_1 - \frac{a_2}{T + a_3},
$$

where $T = 20$($^{\circ}{\rm C}$) and $a_{1,2,3}$ for a water - 1,4 dioxane
system is given below.

|             | $a_1$     | $a_2$      | $a_3$     |
|:------------|:--------|:---------|:--------|
| Water       | 8.07131 | 1730.63  | 233.426 |
| 1,4 dioxane | 7.43155 | 1554.679 | 240.337 |


The following table lists the measured data. Recall that in a binary system $x_1 + x_2 = 1$.

|$x_1$ | 0.0 | 0.1 | 0.2 | 0.3 | 0.4 | 0.5 | 0.6 | 0.7 | 0.8 | 0.9 | 1.0 |
|:-----|:--------|:---------|:--------|:-----|:-----|:-----|:-----|:-----|:-----|:-----|:-----|
|$p$| 28.1 | 34.4 | 36.7 | 36.9 | 36.8 | 36.7 | 36.5 | 35.4 | 32.9 | 27.7 | 17.5 |

Estimate $A_{12}$ and $A_{21}$ using data from the above table: 

1. Formulate the least square problem; 
2. Since the model is nonlinear, the problem does not have an analytical solution. Therefore, solve it using the gradient descent or Newton's method implemented in HW1; 
3. Compare your optimized model with the data. Does your model fit well with the data?

---

### Problem 2 (50 points) 

Solve the following problem using Bayesian Optimization:
$$
    \min_{x_1, x_2} \quad \left(4-2.1x_1^2 + \frac{x_1^4}{3}\right)x_1^2 + x_1x_2 + \left(-4 + 4x_2^2\right)x_2^2,
$$
for $x_1 \in [-3,3]$ and $x_2 \in [-2,2]$. A tutorial on Bayesian Optimization can be found [here](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/).





In [ ]:
# Problem 1 : Least squares fitting for vapor-liquid equilibrium
import torch
import numpy as np
import matplotlib.pyplot as plt

SMOOTH = 1e-5


def model(X, A, p_sat):
    # Question : Temporary variables and the computational graph ...
    k1 = A[0]*(A[1]*X[1]/(A[0]*X[0] + A[1]*X[1]))
    k2 = A[1]*(A[0]*X[0]/(A[0]*X[0] + A[1]*X[1]))
    t1 = X[0]*torch.exp((k1**2))*p_sat[0]
    t2 = X[1]*torch.exp((k2**2))*p_sat[1]
    return t1 + t2


# Define the variables to be optimized over
A = torch.tensor([2.5,2.8], requires_grad = True, dtype = torch.float64)
# Data
X = np.asarray([np.arange(0,1.1,0.1), 1-np.arange(0,1.1,0.1)])
X = torch.from_numpy(X)
p = torch.tensor([28.1, 34.4, 36.7, 36.9, 36.8, 36.7, 36.5, 35.4, 32.9, 27.8, 17.5], requires_grad =  False, dtype = torch.float64)

# Constants
# a = torch.tensor([[8.07131, 1730.63, 233.426],[7.43155, 1554.679, 240.337]], requires_grad= False)
# T = 20
# p_sat_water = torch.tensor(a[0][0] - (a[0][1]/(T + a[0][2])))
# p_sat_dioxane = torch.tensor(a[1][0] - (a[1][1]/(T + a[1][2])))
p_sat = torch.tensor([1.2424, 1.4598], requires_grad = False, dtype = torch.float64)

alpha = 0.0001
for i in range(500):
    # Define the loss function
    loss = torch.sum((p - model(X, A, p_sat))**2)
    loss.backward()
    print("Alpha:", alpha, "Loss:", loss.data.numpy(), "Gradient:", A.grad.numpy(), "Parameter:", A.data.numpy())
    with torch.no_grad():
        A -= alpha * A.grad
        A.grad.zero_()
    alpha /= 1.1
    
p_final = model(X, A, p_sat)
print("Model: ", p_final)
print("Truth: ", p)


In [ ]:
# Problem 2: Bayesian Optimization
import sklearn.gaussian_process as gp
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import matplotlib.pyplot as plt


def objective(x, mean = 0, var = 0.1):
    '''
    x : 2x1 vector of a sample in the domain of interest
    mean : scalar mean for iid noise term
    var : scalar variance for iid noise term
    '''
    noise = np.random.normal(mean, var)    
    return ((4 - (2.1*(x[0]**2)) + ((x[0]**4)/3))*(x[0]**2)) + (x[0]*x[1]) + ((-4 + 4*x[1]**2)*x[1]**2)




def expected_improvement(X_, X, model, k = 0.01):
    Y_mu, Y_sigma = model.predict(X, return_std = True)
    Y__= model.predict(X_)
    
    # Find the best value of predicted f(x) among the old samples
    with np.errstate(divide='ignore'):
        y_best = np.max(Y_sigma)
        Z = (Y_mu - y_best - k)/Y_sigma
        ei = ((Y_mu - y_best - k)*norm.cdf(Z)) + (Y_sigma*norm.pdf(Z))
        ei[Y_sigma == 0.0] = 0.0
    return X_[np.argmax(ei)]


# optimize the expected improvement
# EI function is highly non linear

x1 = np.random.uniform(-3,3,5)
x2 = np.random.uniform(-2,2,5)
X = np.asarray([[x1],[x2]]).transpose().squeeze(1)
Y = np.asarray([objective(x) for x in X])

kernel = gp.kernels.RBF()
model = gp.GaussianProcessRegressor(kernel=kernel,
                                    alpha=1e-4,
                                    n_restarts_optimizer=0,
                                    normalize_y=True)

n_iters = 10
k = 0.01
for i in range(n_iters):
    model.fit(X,Y)
    # Select next point using expected improvement
    x1 = np.random.uniform(-3,3,5)
    x2 = np.random.uniform(-2,2,5)
    X_ = np.asarray([[x1],[x2]]).transpose().squeeze(1)
    x_next = expected_improvement(X_ = X_, X = X, model = model)
    y_next = objective(x_next)
    X.stack(x_next)
    Y.append(y_next)